# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [9]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [51]:
cur.execute("""SELECT * FROM customers JOIN orders USING (customerNumber) WHERE city = 'Nantes'; """)
df = pd.DataFrame(cur.fetchall())
df.columns = (x[0] for x in cur.description)
df

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000,10123,2003-05-20,2003-05-29,2003-05-22,Shipped,
1,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000,10298,2004-09-27,2004-10-05,2004-10-01,Shipped,
2,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000,10345,2004-11-25,2004-12-01,2004-11-26,Shipped,
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200,10275,2004-07-23,2004-08-02,2004-07-29,Shipped,
4,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200,10315,2004-10-29,2004-11-08,2004-10-30,Shipped,
5,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200,10375,2005-02-03,2005-02-10,2005-02-06,Shipped,
6,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200,10425,2005-05-31,2005-06-07,,In Process,


## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [17]:
# Your code here
cur.execute("""SELECT firstName, lastName FROM employees JOIN offices USING (officeCode) WHERE city = 'Boston'; """)
df = pd.DataFrame(cur.fetchall())
df.columns = (x[0] for x in cur.description)
df.head()

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [21]:
# Your code here
cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices AS o 
               JOIN employees AS e
               ON 
               GROUP BY officeCode
               HAVING n_employees = 0;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [56]:
# WHICH TOP 3 CITIES HAVE PLACED THE MOST ORDERS
# Your code here
cur.execute("""SELECT c.city, COUNT(orderNumber) as num_of_orders
               FROM customers AS c 
               JOIN orders AS o
               USING(customerNumber)
               WHERE country = 'USA'
               GROUP BY city
               ORDER BY num_of_orders DESC
               LIMIT 3;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,city,num_of_orders
0,San Rafael,17
1,NYC,16
2,Brickhaven,8


In [62]:
# WHICH TOP 5 COUNTRIES HAVE PLACE THE MOST ORDERS
cur.execute("""SELECT country, COUNT(orderNumber) as num_of_orders
               FROM customers AS c
               JOIN orders AS o
               USING(customerNumber)
               GROUP BY country
               ORDER BY num_of_orders DESC
               LIMIT 5;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,country,num_of_orders
0,USA,112
1,France,37
2,Spain,36
3,Australia,19
4,New Zealand,15


In [71]:
# RETRIEVE THE PRODUCT NAME AND DESCRIPTION OF THE TOP 5 PRODUCTS WITH THE MOST QUANTITY IN STOCK
cur.execute("""SELECT productDescription, productName, quantityInStock
               FROM products AS p
               JOIN productlines AS p
               USING(productline)
               GROUP BY productName
               ORDER BY quantityInStock DESC
               LIMIT 5;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productDescription,productName,quantityInStock
0,"Official logos and insignias, saddle bags loca...",2002 Suzuki XREO,9997
1,"This model features, opening hood, opening doo...",1995 Honda Civic,9772
2,Official logos and insignias. Working steering...,America West Airlines B757-200,9653
3,The operating parts of this limited edition Di...,2002 Chevy Corvette,9446
4,This model features grille-mounted chrome horn...,1932 Model A Ford J-Coupe,9354


## Level Up: Display the names of every individual product that each employee has sold

In [77]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees AS e
               JOIN customers AS c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

2996


,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang


## Level Up: Display the Number of Products each employee has sold

In [ ]:
# Your code here

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!